In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from selenium import webdriver  
from selenium.webdriver.common.action_chains import ActionChains  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4.element import NavigableString
import time
import re
import requests
import gc

import pandas as pd 
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [5]:
gc.collect()

22

### **1. IPO Data**
* Create IPO dataframe based on each season
* IPO Data processing : 
    * create date range for news searching
    * drop foreign company(KY)

In [4]:
IPO = pd.read_excel('IPO_RawData.xlsx')
IPO = IPO[IPO['ipo_date'] >= '2006-01-01'].reset_index(drop=True)

In [5]:
def clean(df):
    df['stock_code'] = df['name'].str[:4]
    df['name'] = df['name'].str[5:]
    df['age'] = df['ipo_date'] - IPO['establish_date']
    df['age'] = df['age']/timedelta(days=365)
    df['ipo_year'] = df['ipo_date'].dt.year
    df['allot'] = df['allot']*0.01
    df['ipo_month'] = df['ipo_date'].dt.month

clean(IPO)
IPO['KY'] = 0
IPO.loc[IPO['name'].str.contains('KY'),'KY'] = 1

IPO['underprice'] = IPO['close_price']/IPO['offer_price'] - 1

IPO['oversub'] = 1/IPO['allot'] 

# create news search range
s = [] 
e = []
    
for j in IPO['ipo_date']:
    s.append((j - relativedelta(months=1)).strftime("%Y-%m-%d"))
    
for i in IPO['ipo_date']:
    e.append((i- relativedelta(days=1)).strftime("%Y-%m-%d"))

IPO['start_date'] = s
IPO['end_date'] = e

# USE_CODE = IPO[IPO['KY'] == 0][['stock_code']].sort_values(by='stock_code', ascending = 'False').reset_index(drop = True)
IPO_USE = IPO[IPO['KY'] == 0].reset_index(drop = True)

In [6]:
def ipo_group(i,j):
    return IPO_USE[(IPO_USE['ipo_year'] == i) & IPO_USE['ipo_month'].isin(j)].reset_index(drop = True)

IPO_0604, IPO_0608, IPO_0612 = ipo_group(2006,list(range(1,5))), ipo_group(2006,list(range(5,9))), ipo_group(2006,list(range(9,13)))
IPO_0704, IPO_0708, IPO_0712 = ipo_group(2007,list(range(1,5))), ipo_group(2007,list(range(5,9))), ipo_group(2007,list(range(9,13)))
IPO_0804, IPO_0808, IPO_0812 = ipo_group(2008,list(range(1,5))), ipo_group(2008,list(range(5,9))), ipo_group(2008,list(range(9,13)))
IPO_0904, IPO_0908, IPO_0912 = ipo_group(2009,list(range(1,5))), ipo_group(2009,list(range(5,9))), ipo_group(2009,list(range(9,13)))
IPO_1004, IPO_1008, IPO_1012 = ipo_group(2010,list(range(1,5))), ipo_group(2010,list(range(5,9))), ipo_group(2010,list(range(9,13)))
IPO_1104, IPO_1108, IPO_1112 = ipo_group(2011,list(range(1,5))), ipo_group(2011,list(range(5,9))), ipo_group(2011,list(range(9,13)))
IPO_1204, IPO_1208, IPO_1212 = ipo_group(2012,list(range(1,5))), ipo_group(2012,list(range(5,9))), ipo_group(2012,list(range(9,13)))
IPO_1304, IPO_1308, IPO_1312 = ipo_group(2013,list(range(1,5))), ipo_group(2013,list(range(5,9))), ipo_group(2013,list(range(9,13)))
IPO_1404, IPO_1408, IPO_1412 = ipo_group(2014,list(range(1,5))), ipo_group(2014,list(range(5,9))), ipo_group(2014,list(range(9,13)))
IPO_1504, IPO_1508, IPO_1512 = ipo_group(2015,list(range(1,5))), ipo_group(2015,list(range(5,9))), ipo_group(2015,list(range(9,13)))
IPO_1604, IPO_1608, IPO_1612 = ipo_group(2016,list(range(1,5))), ipo_group(2016,list(range(5,9))), ipo_group(2016,list(range(9,13)))
IPO_1704, IPO_1708, IPO_1712 = ipo_group(2017,list(range(1,5))), ipo_group(2017,list(range(5,9))), ipo_group(2017,list(range(9,13)))
IPO_1804, IPO_1808, IPO_1812 = ipo_group(2018,list(range(1,5))), ipo_group(2018,list(range(5,9))), ipo_group(2018,list(range(9,13)))

### **2. WebCrawler Function**
* open news page
* collect news title date name for each company
* collect news content for each company
* save as excel file

In [1]:
def open_udn():
    driver.implicitly_wait(10)
    driver.maximize_window()
    driver.get('https://udndata.com/ndapp/Index')  
    # fixed ip log in
    driver.find_element_by_xpath("/html/body/div[@id='container']/main[@class='index']/div[@class='wrapper clearfix']/aside[@class='service']/div[@class='member']/div[@class='sign-in']/div[@class='sign-in-ip']/a").click()
    time.sleep(1)

In [7]:
# get news_title, date, name; use in get_news() 
def get_title_date_name(x,y):
    # title, append to title list
    for title in x.find_all(class_ = "control-pic"):
        title_list.append(title.a.string.replace("\n",""))
    # date, append to date list
    for date in x.find_all(class_ = "source"):
        date_list.append(date.string[:10])
    # name, append to name list
    for i in range(len(x.find_all(class_ = "control-pic"))):
        name_list.append(y)

In [8]:
# get news content; use in get_news()
def get_content(j):
    #open new news tab 
    new_tab = driver.find_element_by_xpath("/html/body/div/main/div/div[1]/section/div[6]/ul/li["+str(j)+"]/div/h2/a") 
    action = ActionChains(driver)
    action.move_to_element(new_tab).key_down(Keys.CONTROL).click(new_tab).key_up(Keys.CONTROL).perform() # new tab
    time.sleep(3) # wait for new tab 
    driver.switch_to.window(driver.window_handles[1]) # switch to new tab
    time.sleep(1)
    
    # check log in status
    log_in = BeautifulSoup(driver.page_source,"lxml")
    if log_in.find(text = "會員登入") == "會員登入":
        # login again
        driver.find_element_by_xpath("/html/body/div/main/div/div[1]/section/div[3]/a[1]").click()
    elif log_in.find(text = "錯誤訊息") == "錯誤訊息":
        # refresh window
        driver.refresh()
        time.sleep(1)
    else:
        pass
    
    # get content of new tab, append to content list
    bsobj = BeautifulSoup(driver.page_source,"lxml") 
    content = bsobj.find("article")
    content_list.append(content.get_text().replace("\n","").replace("\t","").replace("\u3000","").replace(" ",""))
    time.sleep(2)
    
    # close new tab
    driver.close()
    time.sleep(1)
    
    # switch back to search result page
    driver.switch_to.window(driver.window_handles[0])

In [9]:
def get_news(name,start_date,end_date):
    # change url
    url = driver.current_url
    driver.get(url)

    # input search element
    search_elem = driver.find_element_by_xpath('/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/div[1]/div[1]/input')
    search_elem.send_keys(name) # input company name
    time.sleep(1)
    
    # input search date range
    start = driver.find_element_by_id('datepicker-start')
    start.send_keys(u""+start_date)
    time.sleep(1)
    end = driver.find_element_by_id('datepicker-end')
    end.send_keys(u""+end_date)
    time.sleep(1)
    
    # click on search button
    driver.find_element_by_xpath("/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/button").click()

    # click on show 50 results in one page button
    select = Select(driver.find_element_by_id('sharepage'))
    select.select_by_value("50")
    time.sleep(1)

    # Find news result number
    news = BeautifulSoup(driver.page_source, "lxml")
    time.sleep(1)
    news_num = int(news.find_all(class_= "mark")[1].text)

    # Find news page
    if (news_num/50).is_integer():
        news_page = int(news_num/50) 
    else:
        news_page = int(news_num/50)+1

    ## For loop to change page and get news content
    for i in range(0,news_page+1):
        # no page
        if i == 0:
            pass
        # page 1
        elif i == 1:
            get_title_date_name(news,name)
            for j in range(1,(news_num-(i*50-51))):
                if j <= 50:
                    get_content(j)
        # other page
        else:
            #click to change page
            driver.find_element_by_xpath("/html/body/div[@id='container']/main/div[@class='wrapper clearfix']/div[@id='mainbar']/section[@class='list-news']/div[@class='page-number page-number-web']/a["+str(i+2)+"]").click()
            driver.get(driver.current_url)
            news = BeautifulSoup(driver.page_source, "lxml")
            get_title_date_name(news,name)
            time.sleep(1)
            for j in range(1,(news_num-(i*50-51))):
                if j <= 50:
                    get_content(j)
    
    ### return to search page
    driver.find_element_by_xpath("/html/body/div/header/div[2]/div/div[1]/div/a[2]").click()
    gc.collect()
    time.sleep(1)

In [13]:
def excel(title):
    df = pd.DataFrame({'name':name_list,'date':date_list,'title':title_list,'content':content_list})
    df.to_excel('./news_data/'+title+'.xlsx', index=False)
    return df

* 2006

In [16]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0604['name'], IPO_0604['start_date'], IPO_0604['end_date']):
    get_news(i,j,k)

driver.quit()
df_0604 = excel('0604')

In [118]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2006.05~08
for i,j,k in zip(IPO_0608['name'],IPO_0608['start_date'], IPO_0608['end_date']):
    get_news(i,j,k)

driver.quit()
df_0608 = excel('0608')

In [27]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2006.05~08
for i,j,k in zip(IPO_0612['name'],IPO_0612['start_date'], IPO_0612['end_date']):
    get_news(i,j,k)

driver.quit()
df_0612 = excel('0612')

* 2007

In [157]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0704['name'],IPO_0704['start_date'], IPO_0704['end_date']):
    get_news(i,j,k)

driver.quit()
df_0704 = excel('0704')

In [158]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0708['name'],IPO_0708['start_date'], IPO_0708['end_date']):
    get_news(i,j,k)

driver.quit()
df_0708 = excel('0708')

In [30]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0712['name'],IPO_0712['start_date'], IPO_0712['end_date']):
    get_news(i,j,k)

driver.quit()
df_0712 = excel('0712')

* 2008

In [12]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0804['name'],IPO_0804['start_date'], IPO_0804['end_date']):
    get_news(i,j,k)

driver.quit()
df_0804 = excel('0804')

In [15]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0808['name'],IPO_0808['start_date'], IPO_0808['end_date']):
    get_news(i,j,k)

driver.quit()
df_0808 = excel('0808')

In [32]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0812['name'],IPO_0812['start_date'], IPO_0812['end_date']):
    get_news(i,j,k)

driver.quit()
df_0812 = excel('0812')

* 2009

In [19]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0904['name'],IPO_0904['start_date'], IPO_0904['end_date']):
    get_news(i,j,k)

driver.quit()
df_0904 = excel('0904')

In [20]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0908['name'],IPO_0908['start_date'], IPO_0908['end_date']):
    get_news(i,j,k)

driver.quit()
df_0908 = excel('0908')

In [36]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_0912['name'],IPO_0912['start_date'], IPO_0912['end_date']):
    get_news(i,j,k)

driver.quit()
df_0912 = excel('0912')

* 2010

In [22]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1004['name'],IPO_1004['start_date'], IPO_1004['end_date']):
    get_news(i,j,k)

driver.quit()
df_1004 = excel('1004')

In [23]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1008['name'],IPO_1008['start_date'], IPO_1008['end_date']):
    get_news(i,j,k)

driver.quit()
df_1008 = excel('1008')

In [2]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1012['name'],IPO_1012['start_date'], IPO_1012['end_date']):
    get_news(i,j,k)

driver.quit()
df_1012 = excel('1012')

* 2011

In [25]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1104['name'],IPO_1104['start_date'], IPO_1104['end_date']):
    get_news(i,j,k)

driver.quit()
df_1104 = excel('1104')

In [10]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1108['name'],IPO_1108['start_date'], IPO_1108['end_date']):
    get_news(i,j,k)

driver.quit()
df_1108 = excel('1108')

In [ ]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1112[~IPO_1112['name'].isin(['安心','傳奇'])]['name'],IPO_1112[~IPO_1112['name'].isin(['安心','傳奇'])]['start_date'], IPO_1112[~IPO_1112['name'].isin(['安心','傳奇'])]['end_date']):
    get_news(i,j,k)

driver.quit()
df_1112 = excel('1112')

* 2012

In [ ]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1204['name'],IPO_1204['start_date'], IPO_1204['end_date']):
    get_news(i,j,k)

driver.quit()
df_1204 = excel('1204')

In [ ]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1208['name'],IPO_1208['start_date'], IPO_1208['end_date']):
    get_news(i,j,k)

driver.quit()
df_1208 = excel('1208')

In [39]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1212['name'],IPO_1212['start_date'], IPO_1212['end_date']):
    get_news(i,j,k)

driver.quit()
df_1212 = excel('1212')

* 2013

In [43]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1304['name'],IPO_1304['start_date'], IPO_1304['end_date']):
    get_news(i,j,k)

driver.quit()
df_1304 = excel('1304')

In [47]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1308['name'],IPO_1308['start_date'], IPO_1308['end_date']):
    get_news(i,j,k)

driver.quit()
df_1308 = excel('1308')

In [53]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1312[~IPO_1312['name'].isin(['大量','大江','神準'])]['name'],IPO_1312[~IPO_1312['name'].isin(['大量','大江','神準'])]['start_date'], IPO_1312[~IPO_1312['name'].isin(['大量','大江','神準'])]['end_date']):
    get_news(i,j,k)

driver.quit()
df_1312 = excel('1312')

* 2014

In [54]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1404[~IPO_1404['name'].isin(['數字'])]['name'],IPO_1404[~IPO_1404['name'].isin(['數字'])]['start_date'], IPO_1404[~IPO_1404['name'].isin(['數字'])]['end_date']):
    get_news(i,j,k)

driver.quit()
df_1404 = excel('1404')

In [57]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1408['name'],IPO_1408['start_date'], IPO_1408['end_date']):
    get_news(i,j,k)

driver.quit()
df_1408 = excel('1408')

In [59]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1412['name'],IPO_1412['start_date'], IPO_1412['end_date']):
    get_news(i,j,k)

driver.quit()
df_1412 = excel('1412')

* 2015

In [65]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1504['name'],IPO_1504['start_date'], IPO_1504['end_date']):
    get_news(i,j,k)

driver.quit()
df_1504 = excel('1504')

In [66]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1508['name'],IPO_1508['start_date'], IPO_1508['end_date']):
    get_news(i,j,k)

driver.quit()
df_1508 = excel('1508')

In [67]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1512['name'],IPO_1512['start_date'], IPO_1512['end_date']):
    get_news(i,j,k)

driver.quit()
df_1512 = excel('1512')

* 2016

In [68]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1604['name'],IPO_1604['start_date'], IPO_1604['end_date']):
    get_news(i,j,k)

driver.quit()
df_1604 = excel('1604')

In [69]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1608['name'],IPO_1608['start_date'], IPO_1608['end_date']):
    get_news(i,j,k)

driver.quit()
df_1608 = excel('1608')

In [77]:
IPO_1612_drop = IPO_1612[~IPO_1612['name'].isin(['互動','創業家'])].reset_index(drop = True)

In [79]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1612_drop['name'],IPO_1612_drop['start_date'], IPO_1612_drop['end_date']):
    get_news(i,j,k)

driver.quit()
df_1612 = excel('1612')

* 2017

In [82]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1704['name'],IPO_1704['start_date'], IPO_1704['end_date']):
    get_news(i,j,k)

driver.quit()
df_1704 = excel('1704')

In [69]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1708['name'],IPO_1708['start_date'], IPO_1708['end_date']):
    get_news(i,j,k)

driver.quit()
df_1708 = excel('1708')

In [79]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1712['name'],IPO_1712['start_date'], IPO_1712['end_date']):
    get_news(i,j,k)

driver.quit()
df_1712 = excel('1712')

* 2018

In [82]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1804['name'],IPO_1804['start_date'], IPO_1804['end_date']):
    get_news(i,j,k)

driver.quit()
df_1804 = excel('1804')

In [69]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1808['name'],IPO_1808['start_date'], IPO_1808['end_date']):
    get_news(i,j,k)

driver.quit()
df_1808 = excel('1808')

In [79]:
# drive chrome
driver = webdriver.Chrome('./chromedriver')  
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1812['name'],IPO_1812['start_date'], IPO_1812['end_date']):
    get_news(i,j,k)

driver.quit()
df_1812 = excel('1812')